In [1]:
from utils import *
device = 'cuda:1'
model_name = 'WideResNet'
output_file = f'model_saves/{model_name}.txt'
sys.stdout = open(output_file, "w")

In [2]:
BATCH_SIZE = 128
NUM_EPOCHS = 30

train_dataloader, test_dataloader = dataloader(pin_memory=True, num_workers=16)
model = models.wide_resnet50_2(pretrained=True)
model.fc = nn.Sequential(
    nn.Linear(in_features=2048, out_features=12, bias=True)
)

for param in model.parameters():
    param.requires_grad = False

for param in model.fc.parameters():
    param.requires_grad = True

model = model.to(device)

optimizer = torch.optim.SGD(model.parameters(), lr=0.1, momentum=0.9)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.7)
loss_fn = torch.nn.CrossEntropyLoss()
accuracy_fn = torchmetrics.classification.Accuracy(num_classes=12).to(device)

start_time = timer()

model_results = train(
    device,
    model,
    train_dataloader,
    test_dataloader,
    optimizer,
    loss_fn,
    accuracy_fn,
    NUM_EPOCHS,
    model_name,
    scheduler)

end_time = timer()

print(f'Total training time: {end_time - start_time:.3f} seconds')

  0%|          | 0/30 [00:00<?, ?it/s]

In [3]:
with open(f'model_saves/{model_name}.pkl', 'wb') as f:
    pickle.dump(model_results, f)